![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/misc/Comparing_Models_Notebook.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest

In [ ]:
!pip install "langtest[johnsnowlabs,transformers,spacy]"

# Harness and Its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [2]:
#Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>


| Parameter     | Description |
| - | - |
| **task**      | Task for which the model is to be evaluated (text-classification or ner) |
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |

<br/>
<br/>

# Comparing Models Using NLP Test

With the LangTest library, testing multiple models and comparing them are now possible. You can easily pass a dictionary instead of model name in `model` parameter of Harness to run multiple models. Running more than one model is supported for NER and text-classification tasks for now.

### New "model" Parameter

Instead of giving a model name or instance in the parameter, now you can give list of dictionaries in the following format :



```python

models = [{"model": "ner.dl" , "hub":"johnsnowlabs"} , {"model":"en_core_web_sm", "hub": "spacy"}]

Harness(..., model=models, ...)

```



## Comparing Text Classification Models


### Setup and Configure Harness

We will compare `en.sentiment.imdb.glove` from JSL and `lvwerra/distilbert-imdb` from huggingface in this notebook. We will use imdb sentiments sample csv dataset. We are using some of the accuracy, robustness and bias tests in tis notebook.

In [ ]:
# Load the sample.csv
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/imdb/sample.csv

In [6]:
models = [{"model": "en.sentiment.imdb.glove" , "hub":"johnsnowlabs"} , {"model":"lvwerra/distilbert-imdb", "hub": "huggingface"}]

harness = Harness(task="text-classification", model=models, data={"data_source":'sample.csv'})

sentimentdl_glove_imdb download started this may take some time.
Approximate size to download 8.7 MB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "american_to_british": {
    "min_pass_rate": 0.7
   }
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  },
  "fairness": {
   "min_gender_f1_score": {
    "min_score": 0.6
   }
  },
  "representation": {
   "min_label_representation_count": {
    "min_count": 50
   }
  }
 }
}


In [7]:
harness.configure({
    "tests":{
        "defaults":{"min_pass_rate":0.5},
        "accuracy":{
            "min_macro_f1_score":{"min_score":0.7},
        },
        "robustness":{
            "add_typo":{"min_pass_rate":0.7},
            "lowercase":{"min_pass_rate":0.7},
        },
        "bias":{
            "replace_to_female_pronouns":{"min_pass_rate":0.7},
        }
    }
})

{'tests': {'defaults': {'min_pass_rate': 0.5},
  'accuracy': {'min_macro_f1_score': {'min_score': 0.7}},
  'robustness': {'add_typo': {'min_pass_rate': 0.7},
   'lowercase': {'min_pass_rate': 0.7}},
  'bias': {'replace_to_female_pronouns': {'min_pass_rate': 0.7}}}}

### Generate the testcases
The result of the generate function now has an extra column called model_name which specifies which model is the testcase is for.

In [8]:
harness.generate()

Generating testcases...: 100%|██████████| 3/3 [00:00<00:00, 2819.38it/s]


In [9]:
harness.testcases()

,model_name,category,test_type,original,test_case
0,en.sentiment.imdb.glove,accuracy,min_macro_f1_score,-,macro
1,en.sentiment.imdb.glove,robustness,add_typo,Just as a reminder to anyone just now reading ...,Just as a reminder to anyone just now reading ...
2,en.sentiment.imdb.glove,robustness,add_typo,Like CURSE OF THE KOMODO was for the creature ...,Like CURSE OF THE KOMODO was for the creature ...
3,en.sentiment.imdb.glove,robustness,add_typo,"I think that the costumes were excellent, and ...","I think that the costumes were excellent, and ..."
4,en.sentiment.imdb.glove,robustness,add_typo,This is one of my most favorite movies of all ...,Thos is one of my most favorite movies of all ...
...,...,...,...,...,...
1197,lvwerra/distilbert-imdb,bias,replace_to_female_pronouns,"The opening was a steal from ""Eight-legged Fre...","The opening was a steal from ""Eight-legged Fre..."
1198,lvwerra/distilbert-imdb,bias,replace_to_female_pronouns,"Now don't get me wrong, I love seeing half nak...","Now don't get me wrong, I love seeing half nak..."
1199,lvwerra/distilbert-imdb,bias,replace_to_female_pronouns,"Though I saw this movie dubbed in French, so I...","Though I saw this movie dubbed in French, so I..."
1200,lvwerra/distilbert-imdb,bias,replace_to_female_pronouns,This is one of the best presentations of the 6...,This is one of the best presentations of the 6...


harness.generate() method automatically generates the test cases (based on the provided configuration)

### Running the tests

In [10]:
harness.run()

Running testcases... : 100%|██████████| 601/601 [09:02<00:00,  1.11it/s]


### Generated Results

In [11]:
harness.generated_results()

,model_name,category,test_type,original,test_case,expected_result,actual_result,pass
0,en.sentiment.imdb.glove,accuracy,min_macro_f1_score,-,macro,0.7,0.0,False
1,en.sentiment.imdb.glove,robustness,add_typo,Just as a reminder to anyone just now reading ...,Just as a reminder to anyone just now reading ...,pos,pos,True
2,en.sentiment.imdb.glove,robustness,add_typo,Like CURSE OF THE KOMODO was for the creature ...,Like CURSE OF THE KOMODO was for the creature ...,neg,neg,True
3,en.sentiment.imdb.glove,robustness,add_typo,"I think that the costumes were excellent, and ...","I think that the costumes were excellent, and ...",pos,pos,True
4,en.sentiment.imdb.glove,robustness,add_typo,This is one of my most favorite movies of all ...,Thos is one of my most favorite movies of all ...,pos,pos,True
...,...,...,...,...,...,...,...,...
1197,lvwerra/distilbert-imdb,bias,replace_to_female_pronouns,"The opening was a steal from ""Eight-legged Fre...","The opening was a steal from ""Eight-legged Fre...",NEGATIVE,NEGATIVE,True
1198,lvwerra/distilbert-imdb,bias,replace_to_female_pronouns,"Now don't get me wrong, I love seeing half nak...","Now don't get me wrong, I love seeing half nak...",NEGATIVE,NEGATIVE,True
1199,lvwerra/distilbert-imdb,bias,replace_to_female_pronouns,"Though I saw this movie dubbed in French, so I...","Though I saw this movie dubbed in French, so I...",POSITIVE,POSITIVE,True
1200,lvwerra/distilbert-imdb,bias,replace_to_female_pronouns,This is one of the best presentations of the 6...,This is one of the best presentations of the 6...,POSITIVE,POSITIVE,True


### Final Results

We can call `.report()` which summarizes the results giving information about pass and fail counts and overall test pass/fail status of models and tests.

In [ ]:
harness.report()

## Comparing NER Models


### Setup and Configure Harness

We will compare `ner.dl` from JSL and `en_core_web_sm` from spacy in this notebook. We will use CoNLL 2003 sample conll dataset. We are not providing a config so default config will be used.

In [15]:
# Load a sample.conll
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/conll/sample.conll

--2023-08-18 05:11:09--  https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/conll/sample.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50519 (49K) [text/plain]
Saving to: ‘sample.conll’

sample.conll        100%[===================>]  49.33K  --.-KB/s    in 0.001s  

2023-08-18 05:11:10 (52.2 MB/s) - ‘sample.conll’ saved [50519/50519]



In [16]:
models = [{"model": "ner.dl" , "hub":"johnsnowlabs"} , {"model":"en_core_web_sm", "hub": "spacy"}]

harness = Harness(task="ner", model=models, data={"data_source":'sample.conll'})

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[OK!]
Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "american_to_british": {
    "min_pass_rate": 0.7
   }
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  },
  "fairness": {
   "min_gender_f1_score": {
    "min_score": 0.6
   }
  },
  "representation": {
   "min_label_representation_count": {
    "min_count": 50
   }
  }
 }
}


### Generate the testcases
The result of the generate function now has an extra column called model_name which specifies which model is the testcase is for.

In [17]:
harness.generate()

Generating testcases...: 100%|██████████| 5/5 [00:00<00:00, 7101.77it/s]


In [18]:
harness.testcases()

,model_name,category,test_type,original,test_case
0,ner.dl,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCKY WIN , FHINA IN SURPRI..."
1,ner.dl,robustness,add_typo,Nadim Ladki,Nadim Ladik
2,ner.dl,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIB , United Arab Emirates 1996-12-06"
3,ner.dl,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Jaian began the defence of their Asian Cup tit...
4,ner.dl,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...
...,...,...,...,...,...
1821,en_core_web_sm,representation,min_label_representation_count,-,MISC
1822,en_core_web_sm,representation,min_label_representation_count,-,LOC
1823,en_core_web_sm,representation,min_label_representation_count,-,O
1824,en_core_web_sm,representation,min_label_representation_count,-,PER


harness.generate() method automatically generates the test cases (based on the provided configuration)

### Running the tests

In [19]:
harness.run()

Running testcases... :   0%|          | 0/913 [00:00<?, ?it/s]

Running testcases... : 100%|██████████| 913/913 [00:21<00:00, 42.11it/s]


### Generated Results

This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed. This dataframe has an extra column called model_name when harness is used to compare different models.

In [20]:
harness.generated_results()

,model_name,category,test_type,original,test_case,expected_result,actual_result,pass
0,ner.dl,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCKY WIN , FHINA IN SURPRI...","JAPAN: LOC, CHINA: LOC","JAPAN: LOC, FHINA: ORG",False
1,ner.dl,robustness,add_typo,Nadim Ladki,Nadim Ladik,Nadim Ladki: ORG,Nadim Ladik: PER,False
2,ner.dl,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIB , United Arab Emirates 1996-12-06","AL-AIN: LOC, United Arab Emirates: LOC","AL-AIB: LOC, United Arab Emirates: LOC",True
3,ner.dl,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Jaian began the defence of their Asian Cup tit...,"Japan: LOC, Asian Cup: MISC, Syria: LOC","Jaian: PER, Asian Cup: MISC, Syria: LOC",False
4,ner.dl,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...,"China: LOC, Uzbekistan: LOC","China: LOC, Uzbekistan: LOC",True
...,...,...,...,...,...,...,...,...
1821,en_core_web_sm,representation,min_label_representation_count,-,MISC,50.0,71.0,True
1822,en_core_web_sm,representation,min_label_representation_count,-,LOC,50.0,183.0,True
1823,en_core_web_sm,representation,min_label_representation_count,-,O,50.0,2291.0,True
1824,en_core_web_sm,representation,min_label_representation_count,-,PER,50.0,507.0,True


### Final Results

We can call `.report()` which summarizes the results giving information about pass and fail counts and overall test pass/fail status of models and tests.

In [21]:
harness.report()

test_type,add_typo,american_to_british,min_gender_f1_score,min_label_representation_count,min_micro_f1_score,replace_to_female_pronouns,replace_to_low_income_country
model_name,,,,,,,
en_core_web_sm,0.750000,1.000000,0.670000,1.000000,0.500000,1.000000,0.860000
ner.dl,0.810000,1.000000,1.000000,1.000000,1.000000,1.000000,0.960000
